In [ ]:
import numpy as np
import time
from skimage.segmentation import chan_vese
import tqdm
from tifffile import imread
from skimage.transform import resize

img = imread("/mnt/d/d1/AB-TGF/15_D6.tif")

sizes = [
    (32, 32),
    (64, 64),
    (128, 128),
    (256, 256)
]

def segment(a: np.array):
    return chan_vese(a, mu=0.05, lambda1=1, lambda2=1.3, tol=1e-3, max_num_iter=100, dt=0.5, init_level_set="checkerboard")

num_runs = 10
results = {}

# Resize all slices once for each size
for size in sizes:
    times = []

    # Resize each slice in the image to the target size
    resized_img = np.zeros((img.shape[0], size[0], size[1]), dtype=np.float32)
    for i in range(img.shape[0]):
        resized_img[i, :, :] = resize(img[i, :, :], size, anti_aliasing=True)
    
    for _ in tqdm.tqdm(range(num_runs)):
        start = time.time()
        for i in range(resized_img.shape[0]):
            segment(resized_img[i,:,:])  # Segment the resized image
        end = time.time()
        times.append(end - start)
    
    median_time = np.median(times)
    std_time = np.std(times)
    mean_time = np.mean(times)
    results[size] = (median_time, std_time, mean_time)

for size, (median_time, std_time, mean_time) in results.items():
    print(f"Size {size}: Median Time = {median_time:.4f}s, Std Dev = {std_time:.4f}s, Mean = {mean_time:.4f}")


In [21]:
import numpy as np
import time
from skimage.segmentation import chan_vese
import tqdm
from tifffile import imread
from skimage.transform import resize
import multiprocessing

img = imread("/mnt/d/d1/AB-TGF/15_D6.tif")

sizes = [
    # (32, 32),
    # (64, 64),
    # (128, 128),
    (256, 256)
]

def segment(a: np.array):
    return chan_vese(a, mu=0.05, lambda1=1, lambda2=1.3, tol=1e-3, max_num_iter=100, dt=0.5, init_level_set="checkerboard")

def segment_image(slice_img):
    return segment(slice_img)

num_runs = 100
results = {}

# Resize all slices once for each size
for size in sizes:
    times = []

    # Resize each slice in the image to the target size
    resized_img = np.zeros((img.shape[0], size[0], size[1]), dtype=np.float32)
    for i in range(img.shape[0]):
        resized_img[i, :, :] = resize(img[i, :, :], size, anti_aliasing=True)
    
    for _ in tqdm.tqdm(range(num_runs)):
        start = time.time()
        
        # Use multiprocessing to process slices in parallel
        with multiprocessing.Pool() as pool:
            pool.map(segment_image, resized_img)  # Process all slices in parallel
        
        end = time.time()
        times.append(end - start)
    
    median_time = np.median(times)
    std_time = np.std(times)
    mean_time = np.mean(times)
    results[size] = (median_time, std_time, mean_time)

for size, (median_time, std_time, mean_time) in results.items():
    print(f"Size {size}: Median Time = {median_time:.4f}s, Std Dev = {std_time:.4f}s, Mean = {mean_time:.4f}")


100%|██████████| 100/100 [05:11<00:00,  3.11s/it]

Size (256, 256): Median Time = 3.2020s, Std Dev = 0.3842s, Mean = 3.1096
